# 翻后权益分析

分析不同公共牌面下的胜率变化，观察听牌如何影响权益。

In [ ]:
from holdem_lab import (
    parse_cards, format_cards,
    EquityRequest, PlayerHand, calculate_equity,
    evaluate_hand,
)
import matplotlib.pyplot as plt

## 三条 vs 同花听牌

In [ ]:
# 翻牌时三条 vs 同花听牌
hole1 = parse_cards("Qh Qd")  # Q三条
hole2 = parse_cards("Ah Kh")  # 坚果同花听牌
board = parse_cards("Qs 7h 2h")  # 公共牌有Q，两张红桃

print(f"玩家 1: {format_cards(hole1)} (Q三条)")
print(f"玩家 2: {format_cards(hole2)} (坚果同花听牌)")
print(f"公共牌: {format_cards(board)}")
print()

# 当前手牌评估
print("当前手牌:")
print(f"  P1: {evaluate_hand(list(hole1) + list(board)).describe()}")
print(f"  P2: {evaluate_hand(list(hole2) + list(board)).describe()}")
print()

# 计算权益
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(hole1)),
        PlayerHand(hole_cards=tuple(hole2)),
    ],
    board=list(board),
    num_simulations=10000,
    seed=42,
)
result = calculate_equity(request)

print("权益:")
print(f"  三条: {result.players[0].equity:.1%}")
print(f"  同花听牌: {result.players[1].equity:.1%}")

## 成手 vs 组合听牌

In [ ]:
# 超对 vs 组合听牌 (同花 + 顺子听牌)
hole1 = parse_cards("Ah Ad")  # 超对
hole2 = parse_cards("Jh Th")  # 组合听牌
board = parse_cards("9h 8c 2h")  # 同花听牌 + 两头顺子听牌

print(f"玩家 1: {format_cards(hole1)} (超对)")
print(f"玩家 2: {format_cards(hole2)} (组合听牌)")
print(f"公共牌: {format_cards(board)}")
print()

# 计算权益
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(hole1)),
        PlayerHand(hole_cards=tuple(hole2)),
    ],
    board=list(board),
    num_simulations=10000,
    seed=42,
)
result = calculate_equity(request)

print("权益:")
print(f"  超对: {result.players[0].equity:.1%}")
print(f"  组合听牌: {result.players[1].equity:.1%}")
print("\n(组合听牌的权益可能比预期高很多！)")

## 各街权益变化

观察从翻牌到转牌再到河牌，权益如何变化。

In [ ]:
# 追踪各街权益变化
hole1 = parse_cards("Kh Kd")  # 超对
hole2 = parse_cards("Ah Qh")  # 同花听牌
flop = parse_cards("Jh 7h 2c")
turn = parse_cards("4s")
river = parse_cards("3d")

streets = [
    ("翻牌", list(flop)),
    ("转牌", list(flop) + list(turn)),
    ("河牌", list(flop) + list(turn) + list(river)),
]

print(f"主角: {format_cards(hole1)} (KK)")
print(f"对手: {format_cards(hole2)} (同花听牌)")
print()

equities = []
for street_name, board in streets:
    print(f"{street_name}: {format_cards(board)}")
    
    request = EquityRequest(
        players=[
            PlayerHand(hole_cards=tuple(hole1)),
            PlayerHand(hole_cards=tuple(hole2)),
        ],
        board=board,
        num_simulations=10000,
        seed=42,
    )
    result = calculate_equity(request)
    eq1 = result.players[0].equity
    eq2 = result.players[1].equity
    equities.append((street_name, eq1, eq2))
    
    hand1 = evaluate_hand(list(hole1) + board)
    hand2 = evaluate_hand(list(hole2) + board)
    print(f"  KK: {eq1:.1%} ({hand1.describe()})")
    print(f"  AQh: {eq2:.1%} ({hand2.describe()})")
    print()

In [ ]:
# 绘制各街权益变化
fig, ax = plt.subplots(figsize=(8, 5))

street_names = [e[0] for e in equities]
eq1 = [e[1] * 100 for e in equities]
eq2 = [e[2] * 100 for e in equities]

x = range(len(street_names))
ax.plot(x, eq1, 'o-', label='KK', color='steelblue', linewidth=2, markersize=10)
ax.plot(x, eq2, 'o-', label='AQh', color='coral', linewidth=2, markersize=10)

ax.set_xticks(x)
ax.set_xticklabels(street_names)
ax.set_ylabel('权益 (%)')
ax.set_title('各街权益变化')
ax.legend()
ax.set_ylim(0, 100)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 牌面结构分析

In [ ]:
# 相同手牌，不同牌面结构
hole1 = parse_cards("Ah Ad")  # AA
hole2 = parse_cards("7c 7d")  # 77

boards = [
    ("干燥牌面", "Kc 9s 2h"),
    ("公对牌面", "Kc Ks 2h"),
    ("湿润牌面(同花)", "8h 5h 2h"),
    ("连接牌面", "Jh Ts 9c"),
    ("77中三条", "7h Kc 2s"),
]

print(f"AA vs 77 在不同牌面结构")
print("=" * 50)

results = []
for name, board_str in boards:
    board = parse_cards(board_str)
    
    request = EquityRequest(
        players=[
            PlayerHand(hole_cards=tuple(hole1)),
            PlayerHand(hole_cards=tuple(hole2)),
        ],
        board=list(board),
        num_simulations=10000,
        seed=42,
    )
    result = calculate_equity(request)
    eq1 = result.players[0].equity
    eq2 = result.players[1].equity
    results.append((name, eq1, eq2))
    
    print(f"{name:20} | {board_str:15} | AA: {eq1:.1%} | 77: {eq2:.1%}")

In [ ]:
# 可视化牌面结构影响
fig, ax = plt.subplots(figsize=(10, 6))

names = [r[0] for r in results]
eq_aa = [r[1] * 100 for r in results]
eq_77 = [r[2] * 100 for r in results]

x = range(len(names))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], eq_aa, width, label='AA', color='steelblue')
bars2 = ax.bar([i + width/2 for i in x], eq_77, width, label='77', color='coral')

ax.set_ylabel('权益 (%)')
ax.set_title('AA vs 77: 牌面结构的影响')
ax.set_xticks(x)
ax.set_xticklabels(names, rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 100)

plt.tight_layout()
plt.show()